In [1]:
each_mirror_step_gen_theta_size = 5000
given_data_size = 1000
eta = 1 ###################################################
total_trial_num = 1        # num trial
n_max_mirror_iterations=50       # num mirror steps
patient_max = 200           # max patient
patient_max_mirror = 5
stopping_norm = 1e-4
num_samples = 1000         # number of theta generated
verbose = True

pretrain_factor = 1
control_factor_1 = 1
outer_lr_final_factor = 1e0
outer_eta_final_factor = 1e0
import math
outer_lr_factor = pow(math.e, math.log(outer_lr_final_factor)/n_max_mirror_iterations)
outer_eta_factor = pow(math.e, math.log(outer_eta_final_factor)/n_max_mirror_iterations)

In [2]:
!pip install normflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normflows: filename=normflows-1.7.2-py2.py3-none-any.whl size=86917 sha256=8930940a0d67c793053df5d5ac7a433b599c8d1ffa65ac125dad45e0a84363cd
  Stored in directory: /root/.cache/pip/wheels/8a/a4/89/3e09f53a561355c45eccfebeffc07a0e34d36a3f41e3ef68a3
Successfully built normflows


In [3]:
# Import required packages
import torch
import numpy as np
import normflows as nf
import os
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
import torch.nn.functional as F
import torch.distributions as TD
import pandas as pd
import seaborn as sb
import torch.nn as nn
import shutil
import gc
import copy
import time
from scipy.optimize import linear_sum_assignment
from scipy.stats import multivariate_t

In [4]:
torch.manual_seed(114514)
# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
sampled_theta_m = nf.distributions.CircularGaussianMixture(n_modes=4) # nf.distributions.TwoMoons().to(device)

cov_mx = control_factor_1*torch.tensor([[1, 0], [0, 1.]], dtype=torch.double).to(device)


In [5]:
def find_L_n_faster(x, theta_list):
  num_samples = theta_list.shape[0]
  given_data_size = x.shape[0]

  std_normal = TD.MultivariateNormal(
            torch.zeros(num_samples, 2).to(device),
            torch.eye(2).to(device).unsqueeze(0).repeat(num_samples, 1, 1),
            validate_args=False)
  x_data_rep = x.repeat(1,1,num_samples).reshape(given_data_size,num_samples,-1).to(torch.double).to(device)
  prob_mx = torch.exp(std_normal.log_prob(theta_list - x_data_rep)).to(torch.double)
  return -torch.mean(torch.log(torch.mean(prob_mx, dim = 1))).to(torch.double)

def find_first_variation_var_faster(x, theta_list):
  num_samples = theta_list.shape[0]
  given_data_size = x.shape[0]

  std_normal = TD.MultivariateNormal(
            torch.zeros(num_samples, 2).to(device),
            torch.eye(2).to(device).unsqueeze(0).repeat(num_samples, 1, 1),
            validate_args=False)
  x_data_rep = x.repeat(1,1,num_samples).reshape(given_data_size,num_samples,-1).to(device)
  prob_mx = torch.exp(std_normal.log_prob(theta_list - x_data_rep))
  row_mean = torch.transpose(torch.mean(prob_mx, dim = 1).repeat(num_samples, 1), 0, 1)
  first_variation = -torch.mean(prob_mx/row_mean, dim = 0)
  first_variation_var = torch.var(first_variation) # torch.var(first_variation)
  return first_variation_var


def find_first_variation_inner_var_faster(x, theta_list, tau, log_prob_model, log_prob_prev):
  num_samples = theta_list.shape[0]
  given_data_size = x.shape[0]

  std_normal = TD.MultivariateNormal(
            torch.zeros(num_samples, 2).to(device),
            torch.eye(2).to(device).unsqueeze(0).repeat(num_samples, 1, 1),
            validate_args=False)
  x_data_rep = x.repeat(1,1,num_samples).reshape(given_data_size,num_samples,-1).to(device)
  prob_mx = torch.exp(std_normal.log_prob(theta_list - x_data_rep))
  row_mean = torch.transpose(torch.mean(prob_mx, dim = 1).repeat(num_samples, 1), 0, 1)
  first_variation = -torch.mean(prob_mx/row_mean, dim = 0)
  kld_loss_true = torch.mean(log_prob_model).to(torch.double) - torch.mean(log_prob_prev).to(torch.double)
  sec_term = (1/tau) * (log_prob_model - log_prob_prev) # if kld_loss_true.item() >= 0 else torch.tensor([0.0]).to(device)
  input = first_variation  + sec_term
  first_variation_var = torch.var(input) # torch.var(input )
  return first_variation_var


In [84]:
eta = 9
n_max_iterations=5000
FVV1_threshold_init = 0.07 # 0.07  ###################################################
mirror_threshold = 0.05  ###################################################
lr = 2e-4 # try 1e-5
beta2 = 0.99  # 0.99default
a = 20  # decay of lr wrt k
b = 10  # decay of inner threshold wrt k
dist = 1.4 # distance
pretrain_factor = 1
set_seed = 115

In [85]:
Ln_rho_k_list = np.zeros((total_trial_num, n_max_mirror_iterations))
times_2d = np.zeros((total_trial_num, n_max_mirror_iterations))
iterations_2d = np.zeros((total_trial_num, n_max_mirror_iterations))

sampled_theta_m = nf.distributions.TwoMoons().to(device)

for trail_num in range(total_trial_num):
  loss_hist = np.array([])
  torch.manual_seed(set_seed + trail_num)
  np.random.seed(set_seed + trail_num)


  sampled_mean = dist*sampled_theta_m.sample(given_data_size).to(device)

  normal_temp = TD.MultivariateNormal(
        sampled_mean.to(device),
        torch.eye(2).to(device).unsqueeze(0).repeat(given_data_size, 1, 1).to(device),
        validate_args=False)

  given_data = normal_temp.sample().detach().to(device)

  mirror_loss_hist = np.zeros(n_max_mirror_iterations)
  times = np.zeros(n_max_mirror_iterations)
  iterations_1d = np.zeros(n_max_mirror_iterations)

  target_pretrain = TD.MultivariateNormal(
      torch.zeros(2).to(device), pretrain_factor * torch.eye(2).to(device))
  # Define 2D Gaussian base distribution
  base = nf.distributions.DiagGaussian(2, trainable=False)
  # Define list of flows
  num_layers = 10
  flows = []
  for i in range(num_layers):
      # Neural network with two hidden layers having 64 units each
      param_map = nf.nets.MLP([1, 64, 64, 2], init_zeros=True )
      flows.append(nf.flows.AffineCouplingBlock(param_map))
      flows.append(nf.flows.Permute(2, mode='swap'))

  model = nf.NormalizingFlow(base, flows).to(device)
  optimizer1 = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3, betas=(0.9, 0.999))

  for it in tqdm(range(200)):
      optimizer1.zero_grad()

      # Get training samples
      x = target_pretrain.sample((num_samples,)).to(device)
      loss = model.forward_kld(x)

      # Do backprop and optimizer step
      if ~(torch.isnan(loss) | torch.isinf(loss)):
          loss.backward()
          optimizer1.step()

      if it % 100 == 99:
        clear_output(wait=True)
        print('Loss:', loss.item())

  current_mirror_patient = 0
  mirror_min = 5000
  mirror_min_prev = 5000
  for mirror_itr in range(n_max_mirror_iterations):
    optimizer_lr = a * lr/(mirror_itr + a)
    inner_threshold =  b * FVV1_threshold_init/(mirror_itr + b)
    flows = []
    for i in range(num_layers):
        param_map = nf.nets.MLP([1, 64, 64, 2], init_zeros=True )
        flows.append(nf.flows.AffineCouplingBlock(param_map))
        flows.append(nf.flows.Permute(2, mode='swap'))


    model2 = nf.NormalizingFlow(base, flows).to(device)
    optimizer_lr_input = optimizer_lr
    optimizer = torch.optim.Adam(model2.parameters(), lr=optimizer_lr_input, weight_decay=1e-5, betas=(0.9, beta2))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)


    model2.train()
    torch.cuda.synchronize()
    start_epoch = time.time()

    input_eta = (eta*(outer_eta_factor**mirror_itr))
    for it in tqdm(range(n_max_iterations), disable = not verbose):

      optimizer.zero_grad()
      z = base.sample(num_samples)
      log_prob_rho_0 = base.log_prob(z)
      sampled_theta, log_det_model = model2.forward_and_log_det(z)
      log_prob_model = log_prob_rho_0 - log_det_model
      log_prob_prev = model.log_prob(sampled_theta)

      L_n_loss = find_L_n_faster(x=given_data.to(torch.double), theta_list=sampled_theta.to(torch.double))
      kld_loss_true = torch.mean(log_prob_model).to(torch.double) - torch.mean(log_prob_prev).to(torch.double)
      kld_loss = kld_loss_true if kld_loss_true.item() >= 0 else torch.tensor([0.0]).to(device)
      kld_loss = kld_loss if kld_loss.item() <= 500 else torch.tensor([500.0]).to(device)
      loss = L_n_loss    + (1/input_eta)*kld_loss

      # Do backprop and optimizer step
      if ~(torch.isnan(loss) | torch.isinf(loss)):
        loss.backward()
        grads = [param.grad.detach().flatten()
            for param in model2.parameters()
            if param.grad is not None]
        norm = torch.cat(grads).norm()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

      z = base.sample(num_samples)
      log_prob_rho_0 = base.log_prob(z)
      sampled_theta, log_det_model = model2.forward_and_log_det(z)
      log_prob_model = log_prob_rho_0 - log_det_model
      log_prob_prev = model.log_prob(sampled_theta)

      FVV = find_first_variation_inner_var_faster(x=given_data, theta_list = sampled_theta, tau = input_eta, log_prob_model = log_prob_model, log_prob_prev = log_prob_prev)
      FVV2 = find_first_variation_var_faster(x=given_data, theta_list = sampled_theta)
      if verbose:
        if it % 100 == 99:
          # clear_output(wait=True)
          print('trail:',trail_num,'m step:',mirror_itr,'Loss:', loss.item(), ' L_n:', L_n_loss.item(), 'FVV1:',FVV.item(),'norm:', norm.item(), 'FVV2:', FVV2.item())

      if FVV.item() < inner_threshold:
        break

    # After finishing the inner loop
    torch.cuda.synchronize()
    end_epoch = time.time()
    elapsed = end_epoch - start_epoch
    times[mirror_itr] = elapsed

    model.load_state_dict(model2.state_dict())

    iterations_1d[mirror_itr]  = it + 1


    torch.manual_seed(set_seed + trail_num)
    with torch.no_grad():
      z = base.sample(num_samples)
      log_prob_rho_0 = base.log_prob(z)
      generated1, log_porb = model.forward_and_log_det(z)
    # x_all_data[mirror_itr,:,:] = generated1

    L_n_loss_temp = L_n_loss
    mirror_loss_hist[mirror_itr] = L_n_loss_temp.to('cpu').data.numpy()

    FVV_mirror = find_first_variation_var_faster(x=given_data, theta_list = generated1)

    if FVV_mirror.item() < mirror_threshold and (FVV.item() < inner_threshold) and (mirror_itr > 1):
      print("finished")
      break
    if FVV_mirror.item() > mirror_threshold and (FVV.item() > inner_threshold) and (mirror_itr > 1):
      print("fail")
      break


  gc.collect()

100%|██████████| 200/200 [00:03<00:00, 64.33it/s]


Loss: 2.8190982341766357


  2%|▏         | 101/5000 [00:04<04:05, 19.92it/s]

trail: 0 m step: 0 Loss: 4.417191410264533  L_n: 4.2957130910964425 FVV1: 0.19370870292186737 norm: 0.38722464442253113 FVV2: 0.27828943729400635


  4%|▍         | 203/5000 [00:09<03:54, 20.49it/s]

...

trail: 0 m step: 3 Loss: 4.234534443115589  L_n: 4.21093548075563 FVV1: 0.26925578713417053 norm: 1.9813593626022339 FVV2: 0.22742484509944916


 98%|█████████▊| 4902/5000 [03:58<00:04, 19.78it/s]

trail: 0 m step: 3 Loss: 4.231653016651534  L_n: 4.2002219774771365 FVV1: 0.2735019326210022 norm: 1.9147788286209106 FVV2: 0.22169534862041473


100%|██████████| 5000/5000 [04:03<00:00, 20.51it/s]


trail: 0 m step: 3 Loss: 4.22281789180951  L_n: 4.202009539445076 FVV1: 0.24299660325050354 norm: 3.2141263484954834 FVV2: 0.20486657321453094
fail


In [81]:
FVV, FVV_mirror, FVV_mirror.item(), FVV.item()

(tensor(0.0489, device='cuda:0', grad_fn=<VarBackward0>),
 tensor(0.0497, device='cuda:0'),
 0.049671996384859085,
 0.04894685000181198)

In [82]:
sum(sum([iterations_1d > 0]))

4

In [83]:
np.mean(iterations_1d[2:][iterations_1d[2:] > 0])

298.0